In [4]:
import math
import numpy as np

In [3]:
math.comb(3,2)

3

In [3]:
from gaft.components import BinaryIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput
from gaft import GAEngine
import math
import random

In [4]:
    mutation = FlipBitMutation(pm=0.1)
    selection = TournamentSelection()
    crossover = UniformCrossover(pc=0.8, pe=0.5)
    indv = BinaryIndividual(ranges=[(0, 1), (0,1), (0,1)], eps=0.001)
    population = Population(indv_template=indv, size=50).init()
    engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[ConsoleOutput])

In [5]:
distance= random.randint(1,200)
path_loss=35.3+37.6*math.log10(distance)+8
LSCG = 10 **-(path_loss/10)

In [10]:
SSCG= np.random.rayleigh()
H= SSCG * LSCG

In [232]:
@engine.fitness_register
@engine.minimize
def fitness(indv): ###########피트니스 함수를 조건부로 새로짜야함
    
    f,o,p, = indv.solution
    
    pen=0
    BW = 180000
    distance= random.randint(1,200)
    path_loss=35.3+37.6*math.log10(distance)+8
    LSCG = 10 **-(path_loss/10)
    SSCG = np.random.rayleigh()
    H = SSCG * LSCG
    max_SNR = (H * 0.2)/(BW * (10**(-20.4)))
    max_D_rate = (BW * math.log2(1+(H * 0.2)/(BW * (10**(-20.4)))))
    T_delay_min = 32*8/max_D_rate 
    
    if T_delay_min > 0.000125 : ## one time slot 초과시.
        o=1
        p=0
        
        T_local_proc = (330*32)/(125000 * (f))*0.000125    
        if T_local_proc < 0.001  ## local 처리시 time 만족한다면.
            done_time = false
            return 
        
    
    f=  f+0.000001
    o = o+0.000001
    p = p+0.000001
    
    BW = 180000
    global H
    distance= random.randint(1,200)
    path_loss=35.3+37.6*math.log10(distance)+8
    LSCG = 10 **-(path_loss/10)
    SSCG = np.random.rayleigh()
    H = SSCG * LSCG
    SNR_u = (H * 0.2* p )/(BW * (10**(-20.4)))
    D_rate = (BW * math.log2(1+(H * 0.2* p )/(BW * (10**(-20.4)))))
    T_delay = 32*8/D_rate
    
    
    ################### T_delay가 0.000125초 미만으로 들어오면 오프로딩.
    ###################그렇지 못할경우 로컬처리는 어떠한가?
    ##################로컬처리라면 o = 1로바뀌고, p=0으로정한다.
    
    global done_error
    done_error=False
    global done_time
    done_time=False
     
    if o> 0.000001 :## 로컬처리비율이 존재할때
        T_local_proc = (330*32)/(125000 * (f))   
        D = math.ceil(T_local_proc)
        q = o*0.1
        roh = q*D
        ii = math.ceil(8-T_local_proc) 
        jj = math.floor(ii/D)
        SIGMA = 0
        for k in range(jj + 1):
            SIGMA = SIGMA + ((q * ((1 - q) ** (D - 1))) ** k) * ((-1) ** k) * math.comb(ii + k - k * D, k)
        error_u = 1 - ((1 - q) ** (-ii - 1)) * (1 - roh) * SIGMA  # find error
        if error_u > 10**-7:
            pen = 100 * (error_u-10**-7)
            done_error = True
    
            
    
    if T_delay > 0.000125 :  ## if transmission delay is larger than time slot
        pen = 10+1000*(T_delay-0.000125)
        done_time = True
        
    global done_urllc
    done_urllc = done_time or done_error
    
    
    
    
    solve = (10**(-15))*((125000*f)**2) * o * (330*32) + (1-o)*(0.2*p)*(32*8)/(D_rate) + pen
    
    return solve

engine.run(ng=200)
best_indv = engine.population.best_indv(engine.fitness)
best_indv.solution
E_consumption = (10**(-15))*((125000*best_indv.solution[0])**2) * best_indv.solution[1] * (330*32) + (1-best_indv.solution[1])*(0.2*best_indv.solution[2])*0.000125


gaft.ConsoleOutput   INFO     Generation number: 200 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 10, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INF

gaft.ConsoleOutput   INFO     Generation: 91, best fitness: -0.001, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 92, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 93, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 94, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 95, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 96, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 97, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 98, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 99, best fitness: -0.001, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 100, best fitness: -0.001, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 101, best fitness: -0.001, scaled fitness: -0.0

gaft.ConsoleOutput   INFO     Generation: 181, best fitness: -0.001, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 182, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 183, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 184, best fitness: -0.001, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 185, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 186, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 187, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 188, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 189, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 190, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 191, best fitness: -0.000, scaled fitn

In [233]:
best_indv = engine.population.best_indv(engine.fitness)

In [234]:
best_indv.solution

[0.681640625, 0.0, 0.2109375]

In [235]:
engine.fitness(best_indv)

-4.99477362041591e-06

In [236]:
E_consumption

5.2734375e-06

In [237]:
done_urllc

False

In [238]:
H

2.2961474228093125e-12

In [239]:
done_time


False

In [240]:
done_error

False

# URLLC GA, local processing part

In [40]:
    mutation = FlipBitMutation(pm=0.1)
    selection = TournamentSelection()
    crossover = UniformCrossover(pc=0.8, pe=0.5)
    indv = BinaryIndividual(ranges=[(0, 1)], eps=0.001)
    population = Population(indv_template=indv, size=50).init()
    engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[ConsoleOutput])

In [87]:
@engine.fitness_register
@engine.minimize

def fitness(indv): ###########피트니스 함수를 조건부로 새로짜야함
    
    global done_time
    done_time = False
    global done_error
    done_error = False
    
    f, = indv.solution
    
    pen = 0
    q = 0.1
    D = math.ceil((330*32)/(125000*(f+0.00001)))
    roh = q*D    
    ii = math.ceil(8-D)
    jj = math.floor(ii/D)
    SIGMA = 0
    error_u = 0
        
    for k in range(jj + 1):
        SIGMA = SIGMA + ((q * ((1 - q) ** (D - 1))) ** k) * ((-1) ** k) * math.comb(ii + k - k * D, k)
        error_u = 1 - ((1 - q) ** (-ii - 1)) * (1 - roh) * SIGMA  # find error
        
    if D > 8: # 8 time slot = 0.001 second
        pen = 5+100*(D - 8) 
        done_time = True
        
    if error_u > 10**-7:
        pen = 5+ 100*(error_u - 10**-7)
        done_error = True
        
    return (10**(-15))*((125000*f)**2)*(330*32) + pen

engine.run(ng=50)

best_indv = engine.population.best_indv(engine.fitness)
f = best_indv.solution[0]
Off_bit = 0
T_d = 0
done = done_time or done_error
EE = (10**(-15))*((125000*f)**2) * (330*32)
        

gaft.ConsoleOutput   INFO     Generation number: 50 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -0.000, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -0.000, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -0.000, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO     Generation: 10, best fitness: -0.001, scaled fitness: -0.001
gaft.ConsoleOutput   INFO

In [88]:
EE

0.0012185668945312502

In [89]:
done_error

False

In [83]:
done_time

False

# URLLC offloading case (good channel gain)

In [ ]:
(0, 1), (0,1), (0,1)

In [5]:
    mutation = FlipBitMutation(pm=0.1)
    selection = TournamentSelection()
    crossover = UniformCrossover(pc=0.8, pe=0.5)
    indv = BinaryIndividual(ranges=[(0, 1), (0,1), (0,1)], eps=0.001)
    population = Population(indv_template=indv, size=50).init()
    engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[ConsoleOutput])

In [191]:
import math

In [ ]:
 distance= random.randint(1,200)
    path_loss=35.3+37.6*math.log10(distance)+8
    LSCG = 10 **-(path_loss/10)
    SSCG = np.random.rayleigh()
    H = SSCG * LSCG
    SNR_u = (H * 0.2* p )/(BW * (10**(-20.4)))
    D_rate = (BW * math.log2(1+(H * 0.2* p )/(BW * (10**(-20.4)))))
    T_delay = 32*8/D_rate

In [177]:
random.randint(1,5)

5

In [365]:
distance = random.randint(1,200)
path_loss = 35.3 
path_loss=35.3+37.6*math.log10(distance)+8
LSCG = 10 **-(path_loss/10)
SSCG = np.random.rayleigh()
H = SSCG * LSCG
BW = 180000 * 3   # random.randint(1,2)

D_rate_max = (BW * math.log2(1+(H * 0.2 )/(BW * (10**(-20.4)))))

T_delay_max = (32*8)/(D_rate_max)
done_done = False
if T_delay_max > 0.000125:
    done_done = True
    
print(T_delay_max)
print(done_done)
@engine.fitness_register
@engine.minimize
def fitness(indv): ###########피트니스 함수를 조건부로 새로짜야함
    
    global done_time
    done_time = False
    global done_error
    done_error = False
    global done_t_delay
    done_t_delay = False
    
    
    f,o,p = indv.solution
    e = 10**-7
    f=f+e
    o=o+e
    p=p+e
    
    SNR_u = (H * 0.2* p )/(BW * (10**(-20.4)))
    D_rate = (BW * math.log2(1+(H * 0.2* p )/(BW * (10**(-20.4)))))
    global T_delay
    T_delay = 32*8/D_rate

    pen = 0
    q = 0.1 * o
    
    D = math.ceil((330*32)/(125000*(f)))
    
    roh = q*D    
    
    ii = math.ceil(8-D)
    jj = math.floor(ii/D)
    SIGMA = 0
    error_u = 0
        
    for k in range(jj + 1):
        SIGMA = SIGMA + ((q * ((1 - q) ** (D - 1))) ** k) * ((-1) ** k) * math.comb(ii + k - k * D, k)
        error_u = 1 - ((1 - q) ** (-ii - 1)) * (1 - roh) * SIGMA  # find error
    
    if T_delay > 0.000125:   
        pen =  500 + 100*(T_delay - 0.000125)
        done_t_delay =True
    if D > 8:                    ## 1ms 조건은 만족
        pen = 500+100*(D - 8) 
        done_time = True
        
    if error_u > 10**-7:        ## 로컬 큐잉
        pen = 500+ 100*(error_u - 10**-7)
        done_error = True
        
        
    return (10**(-15))*((125000*f)**2)*o*(330*32) +  (1-o)*p*T_delay  +pen


engine.run(ng=100)
best_indv = engine.population.best_indv(engine.fitness)
f = best_indv.solution[0]
o = best_indv.solution[1]
p = best_indv.solution[2]
Off_bit = (1-o)*32*8
T_d = 0
done = done_time or done_error or done_t_delay

EE = (10**(-15))*((125000*(f))**2)*o*(330*32) +  (1-o)*p*T_delay    

#리턴 Off_task, T_d, EE, data_rate, BW, done 등

0.0001040307115350313
False
gaft.ConsoleOutput   INFO     Generation number: 100 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -0.000, scaled fitness: -500.001
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -0.000, scaled fitness: -500.001
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -0.000, scaled fitness: -500.001
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 10, best fitness: -0.000, scaled fitnes

gaft.ConsoleOutput   INFO     Generation: 90, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 91, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 92, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 93, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 94, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 95, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 96, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 97, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 98, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 99, best fitness: -0.000, scaled fitness: -0.000
gaft.ConsoleOutput   INFO     Generation: 100, best fitness: -0.000, scaled fitness: -0.00

In [366]:
EE

7.145183457446852e-05

In [367]:
f

0.015625

In [368]:
o

0.0

In [369]:
p

0.572265625

In [370]:
print(done_t_delay)
print(done_time)
print(done_error)


False
False
False


In [364]:
done_done

False

In [385]:
K = np.random.poisson(0.1,100)

In [386]:
np.sum(K, axis=0)

8

In [387]:
K


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

# 갑자기 궁금해서 Energy efficiency로 해봄


In [395]:
distance = random.randint(1,200)
path_loss = 35.3 
path_loss=35.3+37.6*math.log10(distance)+8
LSCG = 10 **-(path_loss/10)
SSCG = np.random.rayleigh()
H = SSCG * LSCG
BW = 180000 * 3   # random.randint(1,2)

D_rate_max = (BW * math.log2(1+(H * 0.2 )/(BW * (10**(-20.4)))))

T_delay_max = (32*8)/(D_rate_max)
done_done = False
if T_delay_max > 0.000125:
    done_done = True
    
print(T_delay_max)
print(done_done)
@engine.fitness_register
def fitness(indv): ###########피트니스 함수를 조건부로 새로짜야함
    
    global done_time
    done_time = False
    global done_error
    done_error = False
    global done_t_delay
    done_t_delay = False
    
    
    f,o,p = indv.solution
    e = 10**-7
    f=f+e
    o=o+e
    p=p+e
    
    SNR_u = (H * 0.2* p )/(BW * (10**(-20.4)))
    D_rate = (BW * math.log2(1+(H * 0.2* p )/(BW * (10**(-20.4)))))
    global T_delay
    T_delay = 32*8/D_rate

    pen = 0
    q = 0.1 * o
    
    D = math.ceil((330*32)/(125000*(f)))
    
    roh = q*D    
    
    ii = math.ceil(8-D)
    jj = math.floor(ii/D)
    SIGMA = 0
    error_u = 0
        
    for k in range(jj + 1):
        SIGMA = SIGMA + ((q * ((1 - q) ** (D - 1))) ** k) * ((-1) ** k) * math.comb(ii + k - k * D, k)
        error_u = 1 - ((1 - q) ** (-ii - 1)) * (1 - roh) * SIGMA  # find error
    
    if T_delay > 0.000125:   
        pen =  -500 + 100*(0.000125-T_delay )
        done_t_delay =True
    if D > 8:                    ## 1ms 조건은 만족
        pen = -500+100*(8-D) 
        done_time = True
        
    if error_u > 10**-7:        ## 로컬 큐잉
        pen = -500+ 100*(10**-7  - error_u )
        done_error = True
        
        
    return (D_rate)/((10**(-15))*((125000*f)**2)*o*(330*32) +  (1-o)*p*T_delay)  + pen


engine.run(ng=300)
best_indv = engine.population.best_indv(engine.fitness)
f = best_indv.solution[0]
o = best_indv.solution[1]
p = best_indv.solution[2]
Off_bit = (1-o)*32*8
T_d = 0
done = done_time or done_error or done_t_delay

#EE = (10**(-15))*((125000*(f))**2)*o*(330*32) +  (1-o)*p*T_delay    

#리턴 Off_task, T_d, EE, data_rate, BW, done 등

9.115516571043924e-05
False
gaft.ConsoleOutput   INFO     Generation number: 300 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -0.011, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -23.269, scaled fitnes

gaft.ConsoleOutput   INFO     Generation: 80, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 81, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 82, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 83, best fitness: -0.004, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 84, best fitness: -50.005, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 85, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 86, best fitness: -0.054, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 87, best fitness: -0.010, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 88, best fitness: -0.010, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 89, best fitness: -0.010, scal

gaft.ConsoleOutput   INFO     Generation: 159, best fitness: -0.011, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 160, best fitness: -5.288, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 161, best fitness: -84479305.000, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 162, best fitness: -0.004, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 163, best fitness: -0.017, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 164, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 165, best fitness: -0.013, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 166, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 167, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 168, best fitne

gaft.ConsoleOutput   INFO     Generation: 238, best fitness: -5.288, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 239, best fitness: -0.043, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 240, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 241, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 242, best fitness: -0.003, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 243, best fitness: -0.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 244, best fitness: -5.001, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 245, best fitness: -0.049, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 246, best fitness: -0.068, scaled fitness: 25619377276308.293
gaft.ConsoleOutput   INFO     Generation: 247, best fitness: -0.

In [397]:
print(done_t_delay)
print(done_time)
print(done_error)


True
True
False


# GA _ eMBB 케이스 Multi-RAT 오프로딩


제약조건 1.T_latency = max {T_local , T_tran + Tproc_mec  } < Tn_max
여기서 최적화시 T_local, T_tran을 유추 가능하지만, T_proc_mec의 경우  MEC의 서비스레이트에 의해 결정되는 부분임.

따라서 T_proc는 deterministic할것이므로, 우리는 T_tran을 최소화하는 방향으로 나아가야함.
그러나 T_tran 을 최소화화면서 동시에  energy consumption을 최소화해야함.
따라서 우리는 이때 조건1을 추가적으로 도입. 
조건1 --> T_tran1 = T_tran2의 조건...

이는 



===번외=======

만약 T_local < T_tran인 경우, Tn_max는 

In [2]:
import random
import math
mutation = FlipBitMutation(pm=0.1)
selection = TournamentSelection()
crossover = UniformCrossover(pc=0.8, pe=0.5)
indv = BinaryIndividual(ranges=[(0, 1), (0,1), (0,1), (0,1), (0,1)], eps=0.001)
population = Population(indv_template=indv, size=50).init()
engine = GAEngine(population=population, selection=selection,crossover=crossover, mutation=mutation,analysis=[ConsoleOutput])

NameError: name 'FlipBitMutation' is not defined

In [136]:
distance1 = random.randint(1,200)
distance2 = random.randint(1,200)
path_loss1=35.3+37.6*math.log10(distance1)+8
path_loss2=35.3+37.6*math.log10(distance2)+8
LSCG1 = 10 **-(path_loss1/10)
LSCG2 = 10 **-(path_loss2/10)
SSCG1 = np.random.rayleigh()
SSCG2 = np.random.rayleigh()

H1 = SSCG1 * LSCG2
H2 = SSCG2* LSCG2
BW1 = 180000 * 1   # random.randint(1,2)
BW2 = 180000 * 1


print(H1)
print(H2)


1.1255848525300918e-12
7.17258373850641e-13


In [144]:

@engine.fitness_register
@engine.minimize
def fitness(indv): ###########피트니스 함수를 조건부로 새로짜야함
    
    global done
    done = False
    
    global T_tran_1
    global T_tran_2
    global T_latency
    
    goodfit=0
    o_1,o_2, f,p1,p2 = indv.solution
    
    pen = 0
    e = 10 ** -7
    
    o0 = 1 * o_1
    o1 = (1 - o_1)*o_2
    o2 = 1 - o1
    
    
    
    
    D1 = (BW1 * math.log2(1+(H1 * 0.1 * (p1) )/(BW1 * (10**(-20.4)))))
    D2 = (BW2 * math.log2(1+(H2 * 0.1 * (p2) )/(BW2 * (10**(-20.4)))))
    
    ran = random.random()
    b_n = 50000*(1+ran)
    Tn_max = 1*(1+ran)
    
    T_local = (330*b_n)/(125000 * (f+e)) * 0.000125
               
    T_tran_1 = (o1*(b_n*8))/(D1 + e)
               
    T_tran_2 = (o2*(b_n*8))/(D2 + e)
               
    T_tran = max(T_tran_1,T_tran_2)
               
    T_latency = max(T_local, T_tran)
               
    
    lam_e = 0.1
    
    if  o0 * lam_e * (330*b_n) - 125000 * f >0:
        pen = 10000 + 1000*(o0 * lam_e * (330*b_n) - 125000 * f)
        done_off = True


    
    if T_latency > Tn_max :
        pen = 10000+100*(T_latency - Tn_max)
        done_time = True

        done = done_time or done_off
    
    ## time latency 동일하게 바꾸기  
    if abs(T_tran_1 - T_tran_2)>= 0.01:
        goodfit = abs(T_tran_1 - T_tran_2)
    
        ##  일단 latency  평등화 ㅇ없이
        
    return (10**(-15))*((125000*f)**2)*o0*(330*b_n) + 0.1*p1*T_tran_1 + 0.1*p2*T_tran_2 + pen +goodfit
 

engine.run(ng=300)
best_indv = engine.population.best_indv(engine.fitness)

o_1 = best_indv.solution[0]
o_2 = best_indv.solution[1]
f  = best_indv.solution[2]
p1 = best_indv.solution[3]
p2 = best_indv.solution[4]
o0 = 1 * o_1
o1 = (1-o_1)*o_2
o2 = 1 - o1

print('T_tran1:',T_tran_1)
print('T_tran2:',T_tran_2)
print('T_latency:',T_latency)
print(done)
print('o0:',o0,'o1:',o1,'o2:',o2,'p1:', f,    'p1:',p1,'p2:',p2)
(10**(-15))*((125000*f)**2)*(1-o1-o2)*(330*b_n) + 0.1*p1*T_tran_1 + 0.1*p2*T_tran_2


gaft.ConsoleOutput   INFO     Generation number: 300 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -0.656, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -3.266, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -5.437, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -1.570, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -1.313, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -2.504, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -8.332, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -7.809, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -3.082, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 10, best fitness: -8.036, scaled fitness: -0.007
gaft.ConsoleOutput   INF

gaft.ConsoleOutput   INFO     Generation: 91, best fitness: -32.731, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 92, best fitness: -0.623, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 93, best fitness: -11.076, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 94, best fitness: -0.538, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 95, best fitness: -8.209, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 96, best fitness: -7.681, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 97, best fitness: -8.006, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 98, best fitness: -8.021, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 99, best fitness: -5.186, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 100, best fitness: -11.350, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 101, best fitness: -3.091, scaled fitness: -

gaft.ConsoleOutput   INFO     Generation: 180, best fitness: -17.759, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 181, best fitness: -34.079, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 182, best fitness: -14.353, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 183, best fitness: -41.009, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 184, best fitness: -30.733, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 185, best fitness: -5.376, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 186, best fitness: -38.547, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 187, best fitness: -12.717, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 188, best fitness: -1.571, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 189, best fitness: -5.131, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 190, best fitness: -15.778, sca

gaft.ConsoleOutput   INFO     Generation: 269, best fitness: -2.983, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 270, best fitness: -2.103, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 271, best fitness: -2.623, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 272, best fitness: -18.105, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 273, best fitness: -34.460, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 274, best fitness: -3.499, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 275, best fitness: -33.830, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 276, best fitness: -4.478, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 277, best fitness: -12.476, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 278, best fitness: -37.534, scaled fitness: -0.007
gaft.ConsoleOutput   INFO     Generation: 279, best fitness: -44.756, scale

0.010971497730504222

In [83]:
o0 =1-o1-o2

In [79]:
o0

0.00390625

In [104]:
(10**(-15))*((125000*f)**2)*(1-o1-o2)*(330*50000)

68.69646749692038

In [10]:

b_n

59298.78441778886